In [1]:
# from src.utils.setCurrDirToBase import setCurrDirToBase
# setCurrDirToBase()

In [2]:
import json
# Scripts
from ta_assignment_automation import parseCourses
from ta_assignment_automation import parseSections
from ta_assignment_automation import parseSchedule
from ta_assignment_automation import combine_course_schedules
from ta_assignment_automation import parseTAData
from ta_assignment_automation import parseInstructorPref
from ta_assignment_automation import computeConflictMatrix
from ta_assignment_automation import create_final_sections
from ta_assignment_automation import duplicateTAs
from ta_assignment_automation import incremental_TA_duplication


ModuleNotFoundError: No module named 'data'

In [ ]:
# Utility files
from ta_assignment_automation import getConflictMatFromTxt
from ta_assignment_automation import showAssignments
from ta_assignment_automation import splitTAhours
from ta_assignment_automation import convertInputFilesToJSON


In [ ]:
# Algorithms
from ta_assignment_automation import hungarian

In [ ]:
convertInputFilesToJSON()

FileNotFoundError: [Errno 2] No such file or directory: 'input_files/TA_schedule.csv'

### Parse Courses

In [ ]:
parseCourses()

Reading Course Data
Writing Course Data to JSON file:  output_files/courses.json
Total Courses added:  80


### Parse Sections

In [ ]:
parseSections()

Reading Course Data to Add Sections
Writing Section Data to JSON file:  output_files/sections.json
Total Sections added:  579


### Parse Course Schedule

In [ ]:
parseSchedule()

Reading Course Data to Add Schedule
Writing Schedule Data to JSON file:  output_files/schedule.json
Total Schedule added:  715


In [ ]:
combine_course_schedules()

Writing FINAL Data to JSON file:  output_files/combined_section_data.json
Total sections added in final data:  220
Writing FINAL Data to JSON file:  output_files/all_section_data.json
Total sections added in final data:  235


In [ ]:
parseTAData()

Reading TA Data
Writing TA Data to JSON file:  output_files/ta_schedule.json
Total TAs added:  53


In [ ]:
parseInstructorPref()

Writing Ins Pref Data to JSON file:  output_files/ins_pref.json
Total preferences added:  42


In [ ]:
create_final_sections()

READING COMBINED SECTION DATA
Processing Lab Data
SAVING SECTION, LAB AND NON-LAB DATA INTO JSON FILES. OUTPUT FOLDER:  output_files/section_data
NO OF TOTAL SECTIONS 134
NO OF FINAL SECTIONS 64


In [ ]:
computeConflictMatrix()

Conflict Breakdown Matrix initialised
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX JSON MAPPING SAVED TO:  output_files/conflict_matrix.json
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX SAVED TO  output_files/conflict_matrix.txt


In [ ]:
conf_matrix = getConflictMatFromTxt()

Conflict matix loaded to use in algo


In [ ]:
assignments = hungarian(conf_matrix, print_results=False)
print(assignments)
showAssignments(assignments, conf_matrix)

([31, 4, 47, 29, 20, 13, 28, 34, 32, 3, 5, 7, 8, 6, 0, 9, 16, 11, 49, 51, 35, 10, 36, 37, 38, 1, 39, 40, 41, 42, 43, 33, 15, 2, 45, 14, 46, 12, 48, 30, 18, 50, 44, 52, -1, 17, 19, 21, -1, -1, -1, 22, -1, 23, -1, -1, 24, -1, 25, -1, 26, 27, -1, -1, 52], array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   4.,   8.,  12.,  16.,  20.,
        24.,  28.,  32.,  36.,  40.,  44.,  48.,  52.,  56.,  60.,  64.,
        68.,  72.,  76.,  80.,  84.,  88.,  92.,  96., 100.]))
job 31 assigned to worker 0 with cost (j = 31, w = 0) = 4
job 4 assigned to worker 1 with cost (j = 4, w = 1) = 4
job 47 assigned to worker 2 with cost (j = 47, w = 2) = 4
job 29 assigned to worker 3 with cost (j = 29, w = 3) = 0
job 20 assigned to worker 4 with cost (j = 20, w = 4) = 0
job 13 assigned to worker 5 with cost (j = 13, w = 5) = 0
job 28 assigned to worker 6 with cost (j = 28,

In [ ]:
# WE NEED TO DUPLICATE TAs WHO HAVE BEEN ASSIGNED TO CLASS LESS THAN 40 (CAPACITY_CAP) ENROLLMENT

duplicateTAs()

TOTAL TAs before Duplication:  53
TOTAL TAs after Duplication: 
63



In [ ]:
# RUN THE HUNGARIAN ON THE NEW CONFLICT MATRIX AGAIN
computeConflictMatrix()
conf_matrix = getConflictMatFromTxt()
assignments = hungarian(conf_matrix, print_results=False)
print(assignments)
showAssignments(assignments, conf_matrix, showLogs=False)

Conflict Breakdown Matrix initialised
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX JSON MAPPING SAVED TO:  output_files/conflict_matrix.json
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX SAVED TO  output_files/conflict_matrix.txt
Conflict matix loaded to use in algo
([31, 4, 47, 29, 20, 13, 28, 34, 32, 3, 5, 7, 8, 6, 55, 9, 16, 11, 49, 51, 53, 10, 36, 37, 38, 1, 39, 40, 41, 42, 43, 33, 15, 2, 45, 14, 46, 12, 48, 30, 18, 50, 44, 52, 35, 17, 19, 21, 54, 0, 56, 22, 57, 23, 58, 59, 24, 60, 25, 61, 26, 27, 62, -1, 62], array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   4.,   8.,  12.,  16.,  20.,
        24.,  28.,  32.,  36.,  40.,  44.,  48.,  52.,  56.,  60.,  64.,
        68.,  72.,  76.,  80.,  84.,  88.,  92.,  96., 100., 104., 108.,
       112.,

In [ ]:

with open("assignment_output_files/unassigned_courses.json", 'r') as file:
    unassignedCourses = json.load(file)
while len(unassignedCourses) != 0:
    incremental_TA_duplication()
    computeConflictMatrix()
    conf_matrix = getConflictMatFromTxt()
    assignments = hungarian(conf_matrix, print_results=False)
    print(assignments)
    showAssignments(assignments, conf_matrix, showLogs=False)
    with open("assignment_output_files/unassigned_courses.json", 'r') as file:
        unassignedCourses = json.load(file)


INCREMENTAL DUPLICATION: 
NO OF TAs: 64
Conflict Breakdown Matrix initialised
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX JSON MAPPING SAVED TO:  output_files/conflict_matrix.json
CONFLICT BREAKDOWN MATRIX SAVED TO: assignment_output_files/cost_break_down.json
CONFLICT MATRIX SAVED TO  output_files/conflict_matrix.txt
Conflict matix loaded to use in algo
([31, 4, 47, 29, 20, 13, 28, 34, 32, 3, 5, 7, 8, 6, 55, 9, 16, 11, 49, 51, 53, 10, 36, 37, 38, 1, 39, 40, 41, 42, 43, 33, 15, 2, 45, 14, 46, 12, 48, 30, 18, 50, 44, 52, 35, 17, 19, 21, 54, 0, 56, 22, 57, 23, 58, 59, 24, 60, 25, 61, 26, 27, 62, 63, 63], array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   4.,   8.,  12.,  16.,  20.,
        24.,  28.,  32.,  36.,  40.,  44.,  48.,  52.,  56.,  60.,  64.,
        68.,  72.,  76.,  80.,  84.,  88.,  9

In [ ]:
showAssignments(assignments, conf_matrix)

job 31 assigned to worker 0 with cost (j = 31, w = 0) = 4
job 4 assigned to worker 1 with cost (j = 4, w = 1) = 4
job 47 assigned to worker 2 with cost (j = 47, w = 2) = 4
job 29 assigned to worker 3 with cost (j = 29, w = 3) = 0
job 20 assigned to worker 4 with cost (j = 20, w = 4) = 0
job 13 assigned to worker 5 with cost (j = 13, w = 5) = 0
job 28 assigned to worker 6 with cost (j = 28, w = 6) = 4
job 34 assigned to worker 7 with cost (j = 34, w = 7) = 4
job 32 assigned to worker 8 with cost (j = 32, w = 8) = 4
job 3 assigned to worker 9 with cost (j = 3, w = 9) = 0
job 5 assigned to worker 10 with cost (j = 5, w = 10) = 0
job 7 assigned to worker 11 with cost (j = 7, w = 11) = 0
job 8 assigned to worker 12 with cost (j = 8, w = 12) = 0
job 6 assigned to worker 13 with cost (j = 6, w = 13) = 0
job 55 assigned to worker 14 with cost (j = 55, w = 14) = 0
job 9 assigned to worker 15 with cost (j = 9, w = 15) = 0
job 16 assigned to worker 16 with cost (j = 16, w = 16) = 0
job 11 assigne

In [ ]:
splitTAhours()

DUPLICATE TAs
------------------------------------------------
fr24721@uga.edu
sb66469@uga.edu
vrb18212@uga.edu
nikhita.biradar@uga.edu
ehsanasali@uga.edu
ruili.fang@uga.edu
aag53521@uga.edu
ap22943@uga.edu
tt59612@uga.edu
ct48249@uga.edu
vg80700@uga.edu
------------------------------------------------
